In [1]:
using BenchmarkTools, Compat, DataFrames, Distributions, ForwardDiff

In [2]:
# Basic trust region with truncated conjugate gradient.

df = readtable("model_australia.txt", separator = ' ', header = false)

head(df)

,x1,x2,x3,x4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30


In [3]:
df2 = readtable("model.txt", separator = ' ', header = false)

names!(df2, [:C, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10, :x11, :x12])

head(df2)

,C,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
1,1,35,0,0,0,69,34,35,0,70,71,70,30
2,1,30,0,0,0,64,44,53,0,68,84,85,50
3,1,40,0,0,0,69,34,35,0,129,195,149,101
4,1,70,0,0,0,64,44,53,0,59,79,81,32
5,4,45,0,0,0,64,44,53,0,82,93,94,99
6,2,20,0,0,0,69,40,35,0,70,57,58,43


In [4]:
mixed_logit = DataFrame(β1 = 1.0:210.0, β2 = 1.0:210.0, β3 = 1.0:210.0, β4 = 1.0:210.0, β5 = 1.0:210.0)

#mixed_logit = DataFrame(β1 = 1.0:210.0, β2 = 1.0:210.0)

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

head(mixed_logit)

,β1,β2,β3,β4,β5
1,1.0,1.0,1.0,1.0,1.0
2,2.0,2.0,2.0,2.0,2.0
3,3.0,3.0,3.0,3.0,3.0
4,4.0,4.0,4.0,4.0,4.0
5,5.0,5.0,5.0,5.0,5.0
6,6.0,6.0,6.0,6.0,6.0


In [5]:
function simulate(k::Int64 = 5)
    for i = 1:210
        for j = 1:k
            mixed_logit[i, j] = rand_contdist(Uniform())
        end
    end
end

simulate()

head(mixed_logit)

,β1,β2,β3,β4,β5
1,0.47929599727351846,0.12945494717443462,0.4768769162852913,0.9274269213740278,0.8449560730963026
2,0.28160004374144987,0.34670400809238244,0.5179264850953413,0.9188134682895739,0.32804645258215426
3,0.8188996941414981,0.6379341055956058,0.479224176620926,0.8773119498511264,0.3700742476383858
4,0.5178828407577598,0.8661199051229402,0.544914269081546,0.2202404954804691,0.9869653925422388
5,0.20764323258395279,0.7294863018449489,0.3150061809545208,0.9318812101529259,0.5154907435918326
6,0.1256974282186203,0.925258940321839,0.08555472952063803,0.5234821903500244,0.30916927721950427


In [6]:
function individual(θ::Vector, i::Int64)
    data = convert(Array, df[i*7-6:i*7-1, :])
    choices = convert(Array, df[i*7:i*7, :])
    alternatives = find(choices .== 0)
    choice = find(choices .== 1)[1]
    
    function utility(β::Vector, i::Int64)
        return dot(vec(data[:, i]), β)
    end
    
    function construct(γ::Vector, θ::Vector, βk::Float64 = 0.0)
        for k = 1:length(γ)
            βk += θ[1]+θ[2]*γ[k]
        end
        return βk
    end
    
    function probability(θ::Vector, t::Float64 = 0.0)
        γ = vec(convert(Array, mixed_logit[i, 1:5]))
        β = θ[1:4]
        push!(β, construct(γ, θ[5:6]))
        push!(β, θ[length(θ)])
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return probability
end

individual (generic function with 1 method)

In [7]:
function f(θ::Vector, model::Float64 = 0.0, n::Int64 = 210)
    i = 1
    while i <= n
        probability = individual(θ, i)
        model += log(probability(θ))
        i += 1
    end
    return -model/n
end

f (generic function with 3 methods)

In [8]:
immutable BasicTrustRegion{T<:Real}
    η1::T
    η2::T
    γ1::T
    γ2::T
end

function BTRDefaults()
    return BasicTrustRegion(0.01, 0.9, 0.5, 0.5)
end

type BTRState
    iter::Int64
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64
    tol::Float64

    function BTRState()
        state = new()
        state.tol = 1e-6
        return state
    end
end

In [9]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η1
        return true
    else
        return false
    end
end

function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η2
        stepnorm = norm(state.step)
        state.Δ = min(1e20, max(4*stepnorm, state.Δ))
    elseif state.ρ >= b.η1
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

In [10]:
function TruncatedCG(g::Vector, H::Matrix, Δ::Float64)
    n = length(g)
    s = zeros(n)
    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g, v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ2 = Δ*Δ
    while stopCG(norm(g), normg0, k, n) == false
        Hd = H*d
        κ = dot(d, Hd)
        if κ <= 0
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        α = gv/κ
        norm2s += α*(2*sMd+α*norm2d)
        if norm2s >= Δ2
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        s += α*d
        g += α*Hd
        v = g
        newgv = dot(g, v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        k += 1
    end
    return s
end

TruncatedCG (generic function with 1 method)

In [11]:
function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int)
    χ::Float64 = 0.1
    θ::Float64 = 0.5
    if (k == kmax) || (normg <= normg0*min(χ, normg0^θ))
        return true
    else
        return false
    end
end

stopCG (generic function with 1 method)

In [12]:
function btr(f::Function, g!::Function, H!::Function, Step::Function, x0::Vector, state::BTRState = BTRState())
    b = BTRDefaults()
    state.iter = 0
    state.x = x0
    n = length(x0)
    tol2 = state.tol*state.tol
    state.g = zeros(n)
    H = eye(n, n)
    fx = f(x0)
    g!(x0, state.g)
    state.Δ = 0.1*norm(state.g)
    y = zeros(n)
    gcand = zeros(n)
    nmax = 100
    
    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while dot(state.g, state.g) > tol2 && state.iter < nmax
        state.step = Step(state.g, H, state.Δ)
        state.xcand = state.x+state.step
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))
        g!(state.xcand, gcand)
        y = gcand-state.g
        H = H!(H, y, state.step)
        if acceptCandidate!(state, b)
            state.x = copy(state.xcand)
            state.g = copy(gcand)
            fx = fcand
        end
        updateRadius!(state, b)
        state.iter += 1
    end
    return state.x, state.iter
end

btr (generic function with 2 methods)

In [13]:
function g(x::Vector, n::Int64 = 210)
    t = zeros(length(x))
    for i = 1:n
        probability = individual(x, i)
        t += (1/probability(x))*ForwardDiff.gradient(probability, x)
    end
    return -t/n
end

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [14]:
function H(x::Vector)
    return ForwardDiff.hessian(f, x)
end

function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [15]:
function BHHH(x::Vector, n::Int64 = 210)
    t = zeros(length(x), length(x))
    for i = 1:n
        probability = individual(x, i)
        g = ForwardDiff.gradient(probability, x)
        t += g*(g')
    end
    return t/n
end

function BHHH!(x::Vector, storage::Matrix)
    s = BHHH(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

BHHH! (generic function with 1 method)

In [16]:
function BFGS(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

function BFGS!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = BFGS(B, y, s)
end

BFGS! (generic function with 1 method)

In [17]:
function SR1(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

function SR1!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = SR1(B, y, s)
end

SR1! (generic function with 1 method)

In [18]:
println(btr(f, g!, BFGS!, TruncatedCG, zeros(7), BTRState()))

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 100)


In [19]:
#btr(f, g!, SR1!, TruncatedCG, zeros(7), BTRState())